# Polar Expression Plugins with `rustimport`

For Google colab, we install [rustimport](https://github.com/mityax/rustimport) and the rust toolchain:

In [ ]:
import os
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install rustimport polars==0.20.2
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] += ":/root/.cargo/bin"

In [ ]:
%load_ext rustimport

## Define the polars expression

As show in [Polar's documentation](https://pola-rs.github.io/polars/user-guide/expressions/plugins/), we can define a pig-latin expression in rust. We'll need the compiled module later, so we set `--module-path-variable=polars_pig_latin_module`:

In [ ]:
%%rustimport --module-path-variable=polars_pig_latin_module
//: [dependencies]
//: pyo3 = { version = "0.20.0", features = ["extension-module"] }
//: pyo3-polars = { version = "0.11.1", features = ["derive"] }
//: serde = { version = "1", features = ["derive"] }
//: polars = { version = "0.37.0", features = ["dtype-struct"], default-features = false }

use polars::prelude::*;
use pyo3::prelude::*;
use pyo3_polars::derive::polars_expr;
use std::fmt::Write;

#[polars_expr(output_type=String)]
fn pig_latinnify(inputs: &[Series]) -> PolarsResult<Series> {
    let ca: &StringChunked = inputs[0].str()?;
    let out: StringChunked = ca.apply_to_buffer(|value: &str, output: &mut String| {
        if let Some(first_char) = value.chars().next() {
            write!(output, "{}{}ay", &value[1..], first_char).unwrap()
        }
    });
    Ok(out.into_series())
}

We install dependencies required by Polars using with a `//:` comment syntax, which will be injected into a `Cargo.toml` file.

With `polars_pig_latin_module` defined, we configure a `language` namespace for the Polars dataframe:

In [ ]:
import polars as pl

def pig_latinnify(expr) -> pl.Expr:
    if isinstance(expr, str):
        expr = pl.col(expr)
    return expr.register_plugin(
        lib=polars_pig_latin_module,
        symbol="pig_latinnify",
        is_elementwise=True,
    )

With the `language` namespace defined, we can now use it with Polars:

In [ ]:
df = pl.DataFrame(
    {
        "convert": ["pig", "latin", "is", "silly"],
    }
)

out = df.with_columns(
    pig_latin=pig_latinnify("convert"),
)
print(out)